### MNIST 分类
- NN 结构: 
- Momentum(动量)优化算法: 梯度下降算法的一种优化
    - 传统的梯度下降, 训练轨迹是锯齿状, 耗时
    - 基础: 指数加权平均法
- Ref: https://gist.github.com/programming-datascience/c9fc99bc01a0c2b6000dfcab76b67551




In [45]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms

In [46]:
# 变量
EPOCHS = 5
BATCH_SIZE = 128
learning_rate = 0.01
momentum = 0.5
MNIST_DIR = '../data'

In [47]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [48]:
def train(model, device, train_loader, optimizer, epoch):
    """
    训练模型
    """
    model.train()
    epoch_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 将变量转移至 MPS
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # 损失函数
        loss.backward()  # 求导
        optimizer.step()
        if batch_idx % 10 > 0:
            continue
        # 训练准确率
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct = pred.eq(target.view_as(pred)).sum().item()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.6f}\tCosts: {:.6f} seconds'.format(
            epoch, 
            batch_idx * len(data), 
            len(train_loader.dataset),
            100. * batch_idx / len(train_loader), 
            loss.item(), 
            correct / len(data),
            time.time() - epoch_time
        ))
        epoch_time = time.time()

In [49]:
def test(model, device, test_loader):
    """
    测试模型
    """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():  # 不需要求导
        for data, target in test_loader:
            # 批量处理
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [50]:
def main():
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)

    device = torch.device("mps")
    print("Using Device: ", device)

    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(MNIST_DIR, train=True, download=False,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(MNIST_DIR, train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)


    model = ConvNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

    for epoch in range(EPOCHS):
        train(model, device, train_loader, optimizer, epoch + 1)
        test(model, device, test_loader)

In [51]:
main()

PyTorch version: 2.0.0
Torchvision version: 0.15.1
Using Device:  mps
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.307433	Accuracy: 0.078125	Costs: 0.075781 seconds
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.233629	Accuracy: 0.250000	Costs: 0.147483 seconds
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.122500	Accuracy: 0.492188	Costs: 0.149870 seconds
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.877282	Accuracy: 0.671875	Costs: 0.135352 seconds
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.537036	Accuracy: 0.765625	Costs: 0.133611 seconds
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.188302	Accuracy: 0.781250	Costs: 0.133775 seconds
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.751606	Accuracy: 0.820312	Costs: 0.132958 seconds
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.492185	Accuracy: 0.921875	Costs: 0.157182 seconds
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.629579	Accuracy: 0.835938	Costs: 0.141460 seconds
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.442261	Accuracy: 0.882812	Costs: 0.139728 seconds
Train Epoch: 1 [

Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.076565	Accuracy: 0.984375	Costs: 0.133943 seconds
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.164195	Accuracy: 0.937500	Costs: 0.132838 seconds
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.071385	Accuracy: 0.984375	Costs: 0.134026 seconds
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.062917	Accuracy: 0.976562	Costs: 0.132996 seconds
Train Epoch: 2 [58880/60000 (98%)]	Loss: 0.147548	Accuracy: 0.953125	Costs: 0.133651 seconds

Test set: Average loss: 0.0901, Accuracy: 9724/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.060102	Accuracy: 0.984375	Costs: 0.024471 seconds
Train Epoch: 3 [1280/60000 (2%)]	Loss: 0.150469	Accuracy: 0.953125	Costs: 0.135193 seconds
Train Epoch: 3 [2560/60000 (4%)]	Loss: 0.057849	Accuracy: 0.984375	Costs: 0.135040 seconds
Train Epoch: 3 [3840/60000 (6%)]	Loss: 0.080613	Accuracy: 0.968750	Costs: 0.133805 seconds
Train Epoch: 3 [5120/60000 (9%)]	Loss: 0.128279	Accuracy: 0.960938	Costs: 0.134131 seconds
Train Epoch: 3 [6400/6

Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.018818	Accuracy: 1.000000	Costs: 0.133547 seconds
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.040406	Accuracy: 0.984375	Costs: 0.132925 seconds
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.072846	Accuracy: 0.968750	Costs: 0.133243 seconds
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.030070	Accuracy: 0.992188	Costs: 0.132984 seconds
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.079619	Accuracy: 0.976562	Costs: 0.133220 seconds
Train Epoch: 4 [52480/60000 (87%)]	Loss: 0.074310	Accuracy: 0.976562	Costs: 0.134839 seconds
Train Epoch: 4 [53760/60000 (90%)]	Loss: 0.066281	Accuracy: 0.992188	Costs: 0.133728 seconds
Train Epoch: 4 [55040/60000 (92%)]	Loss: 0.081101	Accuracy: 0.968750	Costs: 0.133250 seconds
Train Epoch: 4 [56320/60000 (94%)]	Loss: 0.088223	Accuracy: 0.960938	Costs: 0.133615 seconds
Train Epoch: 4 [57600/60000 (96%)]	Loss: 0.023083	Accuracy: 0.992188	Costs: 0.133173 seconds
Train Epoch: 4 [58880/60000 (98%)]	Loss: 0.091610	Accuracy: 0.976562	C

In [52]:
transforms.Normalize?